Importing Libraries

In [1]:
import numpy as np 
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import  GradientBoostingClassifier

Reading Dataset


In [4]:
data=pd.read_csv("D:\gokulnath\ibm\Loan_Predict.csv")
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [5]:
numerical_features = data.select_dtypes(include = [np.number]).columns
categorical_features = data.select_dtypes(include = [np.object]).columns

C:\Users\M.DEVENDRAN\AppData\Local\Temp\ipykernel_21300\3364968343.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features = data.select_dtypes(include = [np.object]).columns


Checking for Null Values

In [6]:
data.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

Replacing Null Values with Mode and Median

In [7]:
data['Gender'] = data['Gender'].fillna(data['Gender'].mode()[0])
data['Married'] = data['Married'].fillna(data['Married'].mode()[0])
data['Dependents'] = data['Dependents'].str.replace('+','')
data['Dependents'] = data['Dependents'].fillna(data['Dependents'].mode()[0])
data['LoanAmount'] = data['LoanAmount'].fillna(data['LoanAmount'].mode()[0])
data['Self_Employed'] = data['Self_Employed'].fillna(data['Self_Employed'].mode()[0])
data['Loan_Amount_Term'] = data['Loan_Amount_Term'].fillna(data['Loan_Amount_Term'].mode()[0])
data['Credit_History'] = data['Credit_History'].fillna(data['Credit_History'].mode()[0])

C:\Users\M.DEVENDRAN\AppData\Local\Temp\ipykernel_21300\1049561728.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['Dependents'] = data['Dependents'].str.replace('+','')


Verifying

In [8]:
data.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

Conversion to Numerical Values

In [9]:
data['Gender'].replace({'Male':1,'Female':0},inplace=True)
data['Dependents'].replace({'0':0,'1':1,'2':2,'3':3},inplace=True)
data['Married'].replace({'Yes':1,'No':0},inplace=True)
data['Self_Employed'].replace({'Yes':1,'No':0},inplace=True)
data['Property_Area'].replace({'Urban':2,'Rural':0,'Semiurban':1},inplace=True)
data['Education'].replace({'Graduate':1,'Not Graduate':0},inplace=True)
data['Loan_Status'].replace({'Y':1,'N':0},inplace=True)

In [10]:
data['CoapplicantIncome']=data['CoapplicantIncome'].astype("int64")
data['LoanAmount']=data['LoanAmount'].astype("int64")
data['Loan_Amount_Term']=data['Loan_Amount_Term'].astype("int64")
data['Credit_History']=data['Credit_History'].astype("int64")

Label Encoding

In [11]:
le = LabelEncoder()
data['Loan_ID'] = le.fit_transform(data.Loan_ID)
data.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,1,0,0,1,0,5849,0,120,360,1,2,1
1,1,1,1,1,1,0,4583,1508,128,360,1,0,0
2,2,1,1,0,1,1,3000,0,66,360,1,2,1
3,3,1,1,0,0,0,2583,2358,120,360,1,2,1
4,4,1,0,0,1,0,6000,0,141,360,1,2,1


In [12]:
smote = SMOTETomek(0.90)

y = data['Loan_Status']
x = data.drop(columns=['Loan_Status'],axis=1)

C:\Users\M.DEVENDRAN\AppData\Roaming\Python\Python39\site-packages\imblearn\utils\_validation.py:586: FutureWarning: Pass sampling_strategy=0.9 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  warnings.warn(


In [13]:
x_bal,y_bal = smote.fit_resample(x,y)
print(y.value_counts())
print(y_bal.value_counts())

1    422
0    192
Name: Loan_Status, dtype: int64
1    362
0    319
Name: Loan_Status, dtype: int64


In [14]:
scaler = StandardScaler()
x_bal = scaler.fit_transform(x_bal)
x_bal = pd.DataFrame(x_bal)
x_bal.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,-1.699989,0.517838,-1.230006,-0.702450,0.637985,-0.327593,0.112805,-0.587119,-0.329522,0.30435,0.581307,1.401119
1,-1.694240,0.517838,0.813004,0.355887,0.637985,-0.327593,-0.116531,-0.060623,-0.227456,0.30435,0.581307,-1.174217
2,-1.688490,0.517838,0.813004,-0.702450,0.637985,3.052570,-0.403292,-0.587119,-1.018466,0.30435,0.581307,1.401119
3,-1.682741,0.517838,0.813004,-0.702450,-1.567435,-0.327593,-0.478832,0.236142,-0.329522,0.30435,0.581307,1.401119
4,-1.676992,0.517838,-1.230006,-0.702450,0.637985,-0.327593,0.140159,-0.587119,-0.061599,0.30435,0.581307,1.401119


Splitting of Data into Train and Test

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x_bal, y_bal, test_size = 0.33, random_state = 30)

Compare The Model

Random Forest 


In [16]:
def randomForestClassifier(x_train, x_test, y_train, y_test):
    rf = RandomForestClassifier() 
    rf.fit(x_train,y_train)
    yPred = rf.predict(x_test)
    print("RandomForestClassifier") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=rf.predict(x_test)
    y_pred1=rf.predict(x_train)
    random_forest_test_accuracy = accuracy_score(y_test,y_pred)
    random_forest_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', random_forest_test_accuracy)
    print('Training accuracy: ',random_forest_train_accuracy)

randomForestClassifier(x_train, x_test, y_train, y_test)

RandomForestClassifier
Confusion matrix
[[ 81  23]
 [ 11 110]]
Classification report
              precision    recall  f1-score   support

           0       0.88      0.78      0.83       104
           1       0.83      0.91      0.87       121

    accuracy                           0.85       225
   macro avg       0.85      0.84      0.85       225
weighted avg       0.85      0.85      0.85       225

Testing accuracy:  0.8488888888888889
Training accuracy:  1.0


Decision Tree 

In [17]:
def decisionTreeClassifier(x_train, x_test, y_train, y_test):
    dt = DecisionTreeClassifier() 
    dt.fit(x_train,y_train)
    yPred = dt.predict(x_test)
    print("DecisionTreeClassifier") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=dt.predict(x_test)
    y_pred1=dt.predict(x_train)
    decision_tree_test_accuracy = accuracy_score(y_test,y_pred)
    decision_tree_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', decision_tree_test_accuracy)
    print('Training accuracy: ', decision_tree_train_accuracy)

decisionTreeClassifier(x_train, x_test, y_train, y_test)

DecisionTreeClassifier
Confusion matrix
[[ 81  23]
 [ 18 103]]
Classification report
              precision    recall  f1-score   support

           0       0.82      0.78      0.80       104
           1       0.82      0.85      0.83       121

    accuracy                           0.82       225
   macro avg       0.82      0.82      0.82       225
weighted avg       0.82      0.82      0.82       225

Testing accuracy:  0.8177777777777778
Training accuracy:  1.0


KNN

In [18]:
def kneighborsClassifier(x_train, x_test, y_train, y_test):
    knn = KNeighborsClassifier() 
    knn.fit(x_train,y_train)
    yPred = knn.predict(x_test)
    print("KNN") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=knn.predict(x_test)
    y_pred1=knn.predict(x_train)
    knn_test_accuracy = accuracy_score(y_test,y_pred)
    knn_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ',knn_test_accuracy)
    print('Training accuracy: ', knn_train_accuracy)

kneighborsClassifier(x_train, x_test, y_train, y_test)

KNN
Confusion matrix
[[ 69  35]
 [ 18 103]]
Classification report
              precision    recall  f1-score   support

           0       0.79      0.66      0.72       104
           1       0.75      0.85      0.80       121

    accuracy                           0.76       225
   macro avg       0.77      0.76      0.76       225
weighted avg       0.77      0.76      0.76       225

Testing accuracy:  0.7644444444444445
Training accuracy:  0.8399122807017544


Xgboost

In [19]:
def xgboost(x_train, x_test, y_train, y_test):
    xg = GradientBoostingClassifier() 
    xg.fit(x_train,y_train)
    yPred = xg.predict(x_test)
    print("Xgboost") 
    print("Confusion matrix")
    print(confusion_matrix(y_test ,yPred) ) 
    print("Classification report")
    print(classification_report (y_test, yPred))
    y_pred=xg.predict(x_test)
    y_pred1=xg.predict(x_train)
    xgboost_test_accuracy = accuracy_score(y_test,y_pred)
    xgboost_train_accuracy = accuracy_score(y_train,y_pred1)
    print('Testing accuracy: ', xgboost_test_accuracy)
    print('Training accuracy: ', xgboost_train_accuracy)
  
xgboost(x_train, x_test, y_train, y_test)

Xgboost
Confusion matrix
[[ 73  31]
 [ 11 110]]
Classification report
              precision    recall  f1-score   support

           0       0.87      0.70      0.78       104
           1       0.78      0.91      0.84       121

    accuracy                           0.81       225
   macro avg       0.82      0.81      0.81       225
weighted avg       0.82      0.81      0.81       225

Testing accuracy:  0.8133333333333334
Training accuracy:  0.9364035087719298


In [20]:
randomForestClassifier(x_train, x_test, y_train, y_test)
decisionTreeClassifier(x_train, x_test, y_train, y_test)
kneighborsClassifier(x_train, x_test, y_train, y_test)
xgboost(x_train, x_test, y_train, y_test)

RandomForestClassifier
Confusion matrix
[[ 78  26]
 [ 14 107]]
Classification report
              precision    recall  f1-score   support

           0       0.85      0.75      0.80       104
           1       0.80      0.88      0.84       121

    accuracy                           0.82       225
   macro avg       0.83      0.82      0.82       225
weighted avg       0.82      0.82      0.82       225

Testing accuracy:  0.8222222222222222
Training accuracy:  1.0
DecisionTreeClassifier
Confusion matrix
[[ 77  27]
 [ 19 102]]
Classification report
              precision    recall  f1-score   support

           0       0.80      0.74      0.77       104
           1       0.79      0.84      0.82       121

    accuracy                           0.80       225
   macro avg       0.80      0.79      0.79       225
weighted avg       0.80      0.80      0.79       225

Testing accuracy:  0.7955555555555556
Training accuracy:  1.0
KNN
Confusion matrix
[[ 69  35]
 [ 18 103]]
Classifi

By comparing the accuracy of all the models we can see that random forest model is giving better accuracy
Hence random forest model is choosen for further steps